# 环境构建

In [ ]:
# 查看当前环境
!echo "当前环境:"
!echo $CONDA_DEFAULT_ENV

In [ ]:
# 创建环境
!conda create --name evalscope python=3.11
!conda init
!source ~/.bashrc
!conda activate evalscope
!conda install jupyterlab
!conda install ipykernel
!python -m ipykernel install --user --name evalscope --display-name "evalscope”

In [ ]:
# 查看当前环境
!echo "当前环境:"
!echo $CONDA_DEFAULT_ENV

In [ ]:
# 安装evalscope和依赖项
!pip install evalscope # 安装 Native backend (默认)
!pip install 'evalscope[opencompass]' # 安装 OpenCompass backend
!pip install 'evalscope[vlmeval]' # 安装 VLMEvalKit backend
!pip install 'evalscope[rag]' # 安装 RAGEval backend
!pip install 'evalscope[perf]' # 安装 模型压测模块 依赖
!pip install 'evalscope[app]' # 安装 可视化 相关依赖

# 性能评估
    采用了EvalScope专门为Qwen3准备的 modelscope/EvalScope-Qwen3-Test 数据集进行评测，会
    围绕模型的推理、指令跟随、代理能力和多语言支持方面能力进行测试，该数据包含 mmlu_pro 、ifeval 、 live_code_bench 、 math_500 、 aime24 等各著名评估数据集。
    数据集地址：https://modelscope.cn/datasets/modelscope/EvalScope-Qwen3-Test/summary

In [ ]:
# 用vllm启动模型
! vllm serve ../model/sft_merge/ --port 8001

In [ ]:
from evalscope import TaskConfig, run_task

task_cfg = TaskConfig(
    model='./Qwen3-1.7B-finetuned-fp16',
    api_url='http://127.0.0.1:8001/v1/chat/completions',
    eval_type='service',
    datasets=[
        'data_collection',
    ],
    dataset_args={
        'data_collection': {
            'dataset_id': 'modelscope/EvalScope-Qwen3-Test',
            'filters': {'remove_until': '</think>'}  # 过滤掉思考的内容
        }
    },
    eval_batch_size=128,
    generation_config={
        'max_tokens': 30000,  # 最大生成token数，建议设置为较大值避免输出截断
        'temperature': 0.6,  # 采样温度 (qwen 报告推荐值)
        'top_p': 0.95,  # top-p采样 (qwen 报告推荐值)
        'top_k': 20,  # top-k采样 (qwen 报告推荐值)
        'n': 1,  # 每个请求产生的回复数量
    },
    timeout=60000,  # 超时时间
    stream=True,  # 是否使用流式输出
    limit=2000,  # 设置为2000条数据进行测试
)

run_task(task_cfg=task_cfg)

In [ ]:
# 查看模型能力评测结果
! evalscope app

# 压力测试

In [ ]:
!evalscope perf  --url "http://127.0.0.1:8001/v1/chat/completions"  --parallel 5 --model ./Qwen3-32B-unsloth-bnb-4bit --number 20 --api openai --dataset openqa --stream